# 1 Import libraries

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time
import datetime
import pytz
import IPython

In [2]:
from pandas import DataFrame
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [3]:
print('TF version:', tf.__version__)
print('GPU devices:', tf.config.list_physical_devices('GPU'))

TF version: 2.8.2
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 2 Data load and preprocessing

In [4]:
from tensorflow.keras.utils import to_categorical

def prepare_data():
  # columns used 
  columns = ['sepal length in cm', 'sepal width in cm', 'petal length in cm', 'petal width in cm', 'class']

	# generate 2d classification dataset
  income = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
                   header=None,
                   names=columns)

  # shuffle arrays because 'class' column is organized in order
  df = shuffle(income.values)
  X = df[:,0:4]
  Y = df[:,4]
	
  # encode class values as integers
  encoder = LabelEncoder()
  encoder.fit(Y)
  encoded_Y = encoder.transform(Y)
  # convert integers to dummy variables (i.e. one hot encoded)
  dummy_y = np_utils.to_categorical(encoded_Y)
	
  # split into train and test
  n_train = 105
  train_x, test_x = X[:n_train, :], X[n_train:, :]
  train_y, test_y = dummy_y[:n_train], dummy_y[n_train:]

  train_x = np.asarray(train_x).astype(np.float32)
  train_y = np.asarray(train_y).astype(np.float32)
  test_x = np.asarray(test_x).astype(np.float32)
  test_y = np.asarray(test_y).astype(np.float32)

  return train_x, train_y, test_x, test_y

In [5]:
# Loading the data 
train_x, train_y, test_x, test_y = prepare_data()

In [6]:
class MyCustomCallback(tf.keras.callbacks.Callback):

  def on_train_begin(self, batch, logs=None):
    self.begins = time.time()
    print('Training: begins at {}'.format(datetime.datetime.now(pytz.timezone('America/Fortaleza')).strftime("%a, %d %b %Y %H:%M:%S")))

  def on_train_end(self, logs=None):
    print('Training: ends at {}'.format(datetime.datetime.now(pytz.timezone('America/Fortaleza')).strftime("%a, %d %b %Y %H:%M:%S")))
    print('Duration: {:.2f} seconds'.format(time.time() - self.begins))    

# 3 Hyperparameter Tuning 

In [7]:
%%capture
!pip install wandb==0.10.17

In [8]:
!wandb login --relogin

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 3.1 Monitoring a neural network

In [9]:
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.callbacks import EarlyStopping

# Default values for hyperparameters
defaults = dict(layer_1 = 8,
                layer_2 = 8,
                learn_rate = 0.01,
                batch_size = 32,
                epoch = 500)

wandb.init(project="projetoii", config= defaults, name="projetoii_run_01")
config = wandb.config

wandb: Currently logged in as: danibmarques (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [10]:
# Instantiate a simple classification model
model = tf.keras.Sequential([
  tf.keras.layers.Dense(config.layer_1, activation=tf.nn.relu, dtype='float32'),
  tf.keras.layers.Dense(config.layer_2, activation=tf.nn.relu, dtype='float32'),
  tf.keras.layers.Dense(3, activation=tf.nn.softmax, dtype='float32')
])

# Instantiate a logistic loss function that expects integer targets.
loss = tf.keras.losses.CategoricalCrossentropy()

# Instantiate an accuracy metric.
accuracy = tf.keras.metrics.CategoricalAccuracy()

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=config.learn_rate)

# configure the optimizer, loss, and metrics to monitor.
model.compile(optimizer=optimizer, loss=loss, metrics=[accuracy])

In [11]:
%%wandb
# Add WandbCallback() to the fit function
model.fit(x=train_x,
          y=train_y,
          batch_size=config.batch_size,
          epochs=config.epoch,
          validation_data=(test_x,test_y),
          callbacks=[WandbCallback(log_weights=True)],
          verbose='auto')


Epoch 1/500
4/4 [==============================] - 3s 72ms/step - loss: 2.7555 - categorical_accuracy: 0.2095 - val_loss: 1.9767 - val_categorical_accuracy: 0.2000
Epoch 2/500
4/4 [==============================] - 0s 15ms/step - loss: 1.5368 - categorical_accuracy: 0.3238 - val_loss: 1.4726 - val_categorical_accuracy: 0.2444
Epoch 3/500
4/4 [==============================] - 0s 16ms/step - loss: 1.2001 - categorical_accuracy: 0.3714 - val_loss: 1.2637 - val_categorical_accuracy: 0.2444
Epoch 4/500
4/4 [==============================] - 0s 19ms/step - loss: 1.0585 - categorical_accuracy: 0.3714 - val_loss: 1.0776 - val_categorical_accuracy: 0.2444
Epoch 5/500
4/4 [==============================] - 0s 17ms/step - loss: 0.9527 - categorical_accuracy: 0.3714 - val_loss: 0.9970 - val_categorical_accuracy: 0.2444
Epoch 6/500
4/4 [==============================] - 0s 15ms/step - loss: 0.9178 - categorical_accuracy: 0.4000 - val_loss: 0.9579 - val_categorical_accuracy: 0.2444
Epoch 7/500
4/4 

## 3.2 Sweeps

In [12]:
 # The sweep calls this function with each set of hyperparameters
def train():
    # Default values for hyper-parameters we're going to sweep over
    defaults = dict(layer_1 = 8,
                layer_2 = 8,
                learn_rate = 0.01,
                batch_size = 32,
                epoch = 500)
    
    # Initialize a new wandb run
    wandb.init(project="projetoii", config= defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
    # Instantiate a simple classification model
    model = tf.keras.Sequential([
                                 tf.keras.layers.Dense(config.layer_1, activation=tf.nn.relu, dtype='float32'),
                                 tf.keras.layers.Dense(config.layer_2, activation=tf.nn.relu, dtype='float32'),
                                 tf.keras.layers.Dense(3, activation=tf.nn.softmax, dtype='float32')
                                 ])

    # Instantiate a logistic loss function that expects integer targets.
    loss = tf.keras.losses.CategoricalCrossentropy()

    # Instantiate an accuracy metric.
    accuracy = tf.keras.metrics.CategoricalAccuracy()

    # Instantiate an optimizer.
    optimizer = tf.keras.optimizers.SGD(learning_rate=config.learn_rate)

    # configure the optimizer, loss, and metrics to monitor.
    model.compile(optimizer=optimizer, loss=loss, metrics=[accuracy])  

    model.fit(train_x, train_y, batch_size=config.batch_size,
              epochs=config.epoch,
              validation_data=(test_x, test_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=100)]
              )   

In [13]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'categorical_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'layer_1': {
            'max': 32,
            'min': 8,
            'distribution': 'int_uniform',
        },
        'layer_2': {
            'max': 32,
            'min': 8,
            'distribution': 'int_uniform',
        },
        'learn_rate': {
            'min': -4,
            'max': -2,
            'distribution': 'log_uniform',  
        },
        'epoch': {
            'values': [300,400,600]
        },
        'batch_size': {
            'values': [32,64]
        }
    }
}

In [14]:
# Initialize a new sweep
# Arguments:
#     – sweep_config: the sweep config dictionary defined above
#     – entity: Set the username for the sweep
#     – project: Set the project name for the sweep
sweep_id = wandb.sweep(sweep_config, entity="danibmarques", project="projetoii")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: hlvbf5fk
Sweep URL: https://wandb.ai/danibmarques/projetoii/sweeps/hlvbf5fk


In [15]:
# Initialize a new sweep
# Arguments:
#     – sweep_id: the sweep_id to run - this was returned above by wandb.sweep()
#     – function: function that defines your model architecture and trains it
wandb.agent(sweep_id = sweep_id, function=train,count=20)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: kcx7b1ea with config:
wandb: 	batch_size: 32
wandb: 	epoch: 600
wandb: 	layer_1: 8
wandb: 	layer_2: 19
wandb: 	learn_rate: 0.08459421447459141
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/600
4/4 [==============================] - 1s 56ms/step - loss: 1.8636 - categorical_accuracy: 0.3238 - val_loss: 0.8581 - val_categorical_accuracy: 0.6000
Epoch 2/600
4/4 [==============================] - 0s 15ms/step - loss: 0.8282 - categorical_accuracy: 0.6095 - val_loss: 0.7005 - val_categorical_accuracy: 0.6000
Epoch 3/600
4/4 [==============================] - 0s 9ms/step - loss: 0.7411 - categorical_accuracy: 0.6000 - val_loss: 0.8645 - val_categorical_accuracy: 0.6000
Epoch 4/600
4/4 [==============================] - 0s 18ms/step - loss: 0.5972 - categorical_accuracy: 0.7048 - val_loss: 0.5197 - val_categorical_accuracy: 0.8000
Epoch 5/600
4/4 [==============================] - 0s 10ms/step - loss: 0.5704 - categorical_accuracy: 0.7905 - val_loss: 0.7476 - val_categorical_accuracy: 0.6000
Epoch 6/600
4/4 [==============================] - 0s 16ms/step - loss: 0.5312 - categorical_accuracy: 0.8000 - val_loss: 0.5810 - val_categorical_accuracy: 0.6000
Epoch 7/600
4/4 [

epoch,586
loss,0.06397
categorical_accuracy,0.99048
val_loss,0.06651
val_categorical_accuracy,0.97778
_step,586
_runtime,40
_timestamp,1658545927
best_val_loss,0.03103
best_epoch,486


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▇▄▃▃▂▃▃▂█▂▄▂▁▂▁▁▁▁▁▂▂▁▁▁▂▁▁▄▁▁▂▁▁▁▁▁▁▂▁▂
categorical_accuracy,▁▆▆▆▇▇▆▇▃▇▆▆█▇███▇█▆▇███▇██▇▇▇▆████████▆
val_loss,▄▃▂▄▄▂▃▃▃▂▂▁▁▁▁▂▁▁▆▄▁▂▂▂▂▁▄▂▁█▁▂▁▁▁▁▁▂▂▁
val_categorical_accuracy,▄▇▆▂▅▇▅▅▆▆▆▇▇▇▇▆▇▇▁▄▇▆▆▇▆▇▅▆█▂▇▇▇▇▇▇▇▆▆▇
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██


wandb: Agent Starting Run: fvjn1i0g with config:
wandb: 	batch_size: 64
wandb: 	epoch: 600
wandb: 	layer_1: 10
wandb: 	layer_2: 22
wandb: 	learn_rate: 0.04647193003946523
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▃▃▃▄▃▄▂▃▃▂▂▄▁▁▃▂▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▂▁▁▁▁▁▁
categorical_accuracy,▁▂▇▆▇▃▄▂█▅▅▆▆▃▇▇▅▆▇███▇██████▅███▆██▇███
val_loss,█▅▄▃▃▆▂▂▂▃▃▂▁▂▁▁▂▁▃▁▁▂▂▂▁▁▁▁▁▃▂▁▁▂▁▁▁▁▁▁
val_categorical_accuracy,▁▆▄█▆▁▇▇█▆▆▇█▇█▇▆█▆█▇▇▇▇█▇▇█▇▆▇██▇█▇▇▇█▇
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██


wandb: Agent Starting Run: u9c3xu6p with config:
wandb: 	batch_size: 64
wandb: 	epoch: 400
wandb: 	layer_1: 19
wandb: 	layer_2: 18
wandb: 	learn_rate: 0.052601905641005295
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/400
2/2 [==============================] - 1s 200ms/step - loss: 1.0673 - categorical_accuracy: 0.3429 - val_loss: 0.8299 - val_categorical_accuracy: 0.7778
Epoch 2/400
2/2 [==============================] - 0s 50ms/step - loss: 0.7660 - categorical_accuracy: 0.8095 - val_loss: 0.7620 - val_categorical_accuracy: 0.7778
Epoch 3/400
2/2 [==============================] - 0s 48ms/step - loss: 0.6982 - categorical_accuracy: 0.8571 - val_loss: 0.7260 - val_categorical_accuracy: 0.7556
Epoch 4/400
2/2 [==============================] - 0s 42ms/step - loss: 0.6693 - categorical_accuracy: 0.7905 - val_loss: 0.6718 - val_categorical_accuracy: 0.8667
Epoch 5/400
2/2 [==============================] - 0s 72ms/step - loss: 0.6661 - categorical_accuracy: 0.8000 - val_loss: 0.6335 - val_categorical_accuracy: 0.8000
Epoch 6/400
2/2 [==============================] - 0s 65ms/step - loss: 0.5966 - categorical_accuracy: 0.8762 - val_loss: 0.6161 - val_categorical_accuracy: 0.7778
Epoch 7/400
2/2

epoch,399
loss,0.09187
categorical_accuracy,0.95238
val_loss,0.05472
val_categorical_accuracy,0.97778
_step,399
_runtime,32
_timestamp,1658546624
best_val_loss,0.04741
best_epoch,397


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▄▆█▄▃▂▃▆▃▂▂▂▆▂▆▂▃▁▁▂▂▁▂▂▂▃▁▁▂▁▄▂▁▂▁▁▁▁
categorical_accuracy,▄▆▇▂▁▆▇█▆▃▆█▇█▃▇▄▇▅██▇▇▇▇█▇▆██▇█▆▇█▆█▇█▇
val_loss,▅▃▃▃▂▃▂▂▂▂▁▁▁▁█▁▁▂▄▁▁▁▂▂▁▁▁▁▁▂▁▂▁▁▂▁▁▂▁▁
val_categorical_accuracy,▆████▄▆▇▇█████▁██▆▃█▇█▇▆█████▇█▇█▇▇██▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██


### 3.2.1 Restore a model

Restore a file, such as a model checkpoint, into your local run folder to access in your script.

See [the restore docs](https://docs.wandb.com/library/restore) for more details.

In [16]:
%%capture
!pip install wandb==0.10.17

In [ ]:
!pip install wandb

In [18]:
 import wandb
 wandb.__version__

'0.10.17'

In [19]:
!wandb login

wandb: Currently logged in as: danibmarques (use `wandb login --relogin` to force relogin)


In [24]:
import wandb
# restore the raw model file "model-best.h5" from a specific run by user "danibmarques"
# in project "projetoii" from run "hlvbf5fk"
best_model = wandb.restore('model-best.h5', run_path="danibmarques/projetoii/dfnec4xm")

In [25]:
# restore the model for tf.keras
model = tf.keras.models.load_model(best_model.name)

In [26]:
# execute the loss and accuracy using the test dataset
loss_, acc_ = model.evaluate(x=test_x,y=test_y, batch_size=64)
print('Test loss: %.3f - acc: %.3f' % (loss_, acc_))

1/1 [==============================] - 0s 120ms/step - loss: 0.0445 - categorical_accuracy: 1.0000
Test loss: 0.044 - acc: 1.000


# 4 References

1. https://github.com/wandb/awesome-dl-projects
2. https://docs.wandb.ai/app/features/panels/parameter-importance
3. https://wandb.ai/wandb/DistHyperOpt/reports/Modern-Scalable-Hyperparameter-Tuning-Methods--VmlldzoyMTQxODM